
Mind Bender
=======

A deep dive into the beast. 
---------------------------------------------


Rob Ludwick -- Boise Python Meetup

June 8, 2015




![QR Code](./chart.png)

![Bender Schematic](./bender_schematic.jpg)

Why?
====

To break through to a higher level of python awareness


Iterators
============


So you're familiar with iterators and list comprehensions.

In [135]:
[ x**2 for x in range(10) ]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [136]:
y = iter(range(3))

try:
    while True:
        print next(y)
except StopIteration:
    print "done"

0
1
2
done


Implementing your own iterator
===============================================


In [137]:
class infinite_zeros(object):
    
    def next(self):
        return 0
    
    def __iter__(self):
        return self

In [138]:
iz = infinite_zeros()

print next(iz)
print iz.next()



0
0


In [139]:
z = iter(iz)
z.next()

0

That's interesting, but not useful.  Let's get the factors of a number.

In [140]:
import math
class factors(object):
    
    def __init__(self, num):
        self.num = num
        self.idx = 0
        
    def next(self):
        while self.idx <= self.num:
            self.idx += 1
            if self.num % self.idx == 0:
                return self.idx
        raise StopIteration
            
    def __iter__(self):
        return self

In [141]:
for x in factors(20):
    print x

1
2
4
5
10
20


In [142]:
for x in factors(17):
    print x

1
17


In [143]:
[z for z in factors(325)]


[1, 5, 13, 25, 65, 325]

So what's the difference between this and a generator?

A generator automatically saves state

In [144]:
def factors(num):
    for idx in range(1, num):
        if num % idx == 0:
            yield idx
    yield num

In [145]:
for x in factors(20):
    print x

1
2
4
5
10
20


So what about the StopIteration?

It's handled automatically by the generator.

In [146]:
y = factors(4)
print next(y)
print next(y)
print next(y)


1
2
4


The next(y) will return a StopIteration.

In [147]:
print next(y)

StopIteration: 

Now let's chain iterators

In [148]:
def square(num_iter):
    while True:
        yield next(num_iter) ** 2

In [149]:
[ x for x in square(factors(20)) ]

[1, 4, 16, 25, 100, 400]

Alternatively:

In [150]:
def square(num_iter):
    for num in num_iter:
        yield num ** 2

In [151]:
[ x for x in square(factors(20)) ]

[1, 4, 16, 25, 100, 400]

Function Objects and Closures
=========

The first step to understanding decorators
---------------

In [152]:
def y(n):
    return n * 2

print type(y)


<type 'function'>


Note the special call() function:

In [153]:
y.__call__(2)

4

In [154]:
def z(f, n):
    return f(n) + 1

Since a function is an object, I can pass a function into another function:

In [155]:
z(y, 2)

5

And because a function is an object I can return a function.

In [156]:
def get_func():
    def func(y):
        return y * 2
    return func


In [157]:
z = get_func()
print z(3)

6


A closure is a function that has access to a variable in an enclosing stack frame after it has exited.

In [158]:
def get_factor_func(num):
    def get_factors():
        z = [x for x in range(1, num+1) if num % x == 0]
        return z
    return get_factors

In [159]:
f = get_factor_func(200)  # This doesn't do anything

In [160]:
f()

[1, 2, 4, 5, 8, 10, 20, 25, 40, 50, 100, 200]

Decorators
=========

Decorators are closures that modify or replace a function's behavior.

In [161]:
def log(f):
    def logger(*args, **kwargs):
        print "{} called with args={}, kwargs={}".format(f.__name__, args, kwargs)
        result = f(*args, **kwargs)
        print "{} returned {}".format(f.__name__, result)
        return result
    return logger



In [162]:
def cube(num):
    return num ** 3

cube = log(cube)

z = cube(3)

cube called with args=(3,), kwargs={}
cube returned 27


In [163]:
@log
def cube(num):
    return num ** 3

z = cube(2)

cube called with args=(2,), kwargs={}
cube returned 8


But we can do this with a class too:

In [164]:
class counter(object):
    """Returns a count of when a function was called."""
    
    def __init__(self, f):
        self.count = 0
        self.f = f
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.f(*args, **kwargs)
    
    def get_count(self):
        return self.count
        


In [165]:
@counter
def cube(num):
    return num ** 3

for idx in range(10):
    z = cube(idx)

print cube.get_count()

10


And there are a few useful cases for this:

In [166]:
class memoize(object):
    
    def __init__(self, f):
        self.cache = {}
        self.f = f
    
    def __call__(self, *args, **kwargs):
        key = (tuple(args), tuple(kwargs))
        if key not in self.cache:
            self.cache[key] = self.f(*args, **kwargs)
        return self.cache[key]
    

import math

@memoize
def is_prime(n):
    """Is n prime?"""
    if n == 1:
        return False
    if n == 2:
        return True
    elif n == 3:
        return True
    else:
        for idx in range(1, int(math.sqrt(n))+1):
            if is_prime(idx):
                if n % idx == 0:
                    return False
        return True

@memoize
def prime(n):
    """Return the nth prime"""
    if n == 1:
        return 2
    else:
        last = prime(n-1)
        primality = False
        while not primality:
            last = last + 1
            primality = is_prime(last)
        return last
    

In [167]:
import time

zero = time.time()
prime(150)
one = time.time()
prime(150)
two = time.time()

print "First Time: {} s".format(one-zero)
print "Second Time: {} s".format(two-one)



First Time: 0.00758695602417 s
Second Time: 6.29425048828e-05 s


In [168]:
prime.cache


{((1,), ()): 2,
 ((2,), ()): 3,
 ((3,), ()): 5,
 ((4,), ()): 7,
 ((5,), ()): 11,
 ((6,), ()): 13,
 ((7,), ()): 17,
 ((8,), ()): 19,
 
 [ ... ] 
 
 ((144,), ()): 827,
 ((145,), ()): 829,
 ((146,), ()): 839,
 ((147,), ()): 853,
 ((148,), ()): 857,
 ((149,), ()): 859,
 ((150,), ()): 863}

In [169]:
is_prime.cache

{((1,), ()): False,
 ((2,), ()): True,
 ((3,), ()): True,
 ((4,), ()): False,
 ((5,), ()): True,
 ((6,), ()): False,
 ((7,), ()): True,
 ((8,), ()): False,
 ((9,), ()): False,
 ((10,), ()): False,
 
 [ ... ] 
 
 ((856,), ()): False,
 ((857,), ()): True,
 ((858,), ()): False,
 ((859,), ()): True,
 ((860,), ()): False,
 ((861,), ()): False,
 ((862,), ()): False,
 ((863,), ()): True}

Types
=====

Let's take a simple class like this:


In [170]:
class my_class(object):
    thing = 5
    
print my_class.thing


5


But we can also express it this way:


In [171]:
my_class = type("my_class", (object,), {"thing": 5})

print my_class.thing


5


Why would we do this?

Instead of writing a code generator, it's easier to just generate the classes we need on the fly.


Syntax
------
<pre>
class_var = type(class_name, subclasses, attributes)
</pre>

These two classes are equivalent:


In [184]:
class counter(object):
    #class level variable
    foo = "bar"
    
    #Initializer
    def __init__(self):
        self.count = 0
    
    def ping(self):
        self.count += 1
    
    def pong(self):
        self.count -= 1

In [185]:

def my_init(self):
    self.count = 0

def my_ping(self):
    self.count += 1

def my_pong(self):
    self.count -= 1

attributes = { "__init__": my_init, "ping": my_ping, "pong": my_pong, "foo": "bar"}
subclasses = (object,)
class_name = "counter2"

counter2 = type(class_name, subclasses, attributes)


In [186]:
c1 = counter()
c1.ping()
c1.ping()
c1.pong()
c1.count

1

In [188]:
c2 = counter2()
c2.ping()
c2.ping()
c2.pong()
c2.count


1

Metaclasses
========

So that's clear I hope.  So what's a metaclass?  It uses the <pre>\_\_metaclass\_\_</pre> attribute on the class like this:

<pre>
class x(object):
    __metaclass__ = metaclass_definition
</pre>

So what's the metaclass definition?

Simply put it's a funtion that takes a type definition and returns another type defintion.

In [202]:
def my_meta(class_name, subclasses, attributes):
    attributes['foo']='bar'
    return type(class_name, subclasses, attributes)

In [203]:
class Nothing(object):
    __metaclass__ = my_meta

In [204]:
Nothing.foo

'bar'

So my adding the metaclass attribute, we can add possibly interesting things to classes (debugging, profiling, abstraction).

To take this one step further, we're going to use a class as our meta:

In [277]:
import time
class MyMeta(type):
    '''Counts the number of times a class has been instantiated'''
    
    def __new__(cls, name, subclasses, attributes):
        """Add attributes to the class before the type instantiation"""

        attributes['cls_ts'] = time.time()
        return type.__new__(cls, name, subclasses, attributes)
    
    def __init__(cls, name, subclasses, attributes):
        """To modify the type after the parent type is created use __init__"""

        def my_init(self, *args, **kwargs):
            self.ts = time.time()
            return super(self.__class__, self).__init__(*args, **kwargs)
        
        def my_repr(self, *args, **kwargs):
            return "<cls_ts: {}, ts: {}>".format(self.cls_ts, self.ts)
        
        cls.__init__ = my_init
        cls.__repr__ = my_repr
        type.__init__(cls, name, subclasses, attributes)
        
        
class Z(object):
    __metaclass__ = MyMeta

for x in range(5):
    time.sleep(1)
    print Z()
                

        

<cls_ts: 1433836359.85, ts: 1433836360.85>
<cls_ts: 1433836359.85, ts: 1433836361.85>
<cls_ts: 1433836359.85, ts: 1433836362.85>
<cls_ts: 1433836359.85, ts: 1433836363.85>
<cls_ts: 1433836359.85, ts: 1433836364.86>


![Applause](./bender_applause.jpg)